In [1]:
from project.dataset import Dataset
import pandas as pd
import nibabel as nib

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.18 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations


In [2]:
ds = Dataset()

### Load all cases

generate slice count for each case using the list comprehension `slices`

In [3]:
mri = ds.load_raw_mri()
masks = ds.load_cmb_masks()
slices = [nib.load(x).get_fdata().shape[2] for x in mri]

pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1

create `standard_df` for the unprocessed data

In [4]:
standard_df = pd.DataFrame({
    'mri': mri,
    'masks': masks,
    'slices': slices
})

function to generate all the target slices for each case

In [5]:
def generate_target_slice(mri, masks, slices):
    if len(mri) != len(masks):
        print(f'Unequal amount of mri cases to cmb masks\t{len(mri)} to {len(masks)}')
    if len(mri) != len(slices):
        print(f'Unequal amount of mri cases to case slice counts\t{len(mri)} to {len(slices)}')

    ls = [(mri[i], masks[i], target) for i in range(len(mri)) for target in range(slices[i])]
    df = pd.DataFrame(ls, columns=['mri', 'masks', 'target'])
    # ls = [(case, target) for case, slices in zip(case, slices) for target in range(slices)]
    return df

In [6]:
df = generate_target_slice(mri, masks, slices)

In [7]:
ar = df.groupby('mri').target.max()
ar

mri
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-101_space-T2S_desc-masked_T2S.nii.gz    34
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-102_space-T2S_desc-masked_T2S.nii.gz    34
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-103_space-T2S_desc-masked_T2S.nii.gz    34
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-104_space-T2S_desc-masked_T2S.nii.gz    34
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-105_space-T2S_desc-masked_T2S.nii.gz    34
                                                                                                                             ..
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-323_space-T2S_desc-masked_T2S.nii.gz    38
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-324_space-T2S_d

In [8]:
arr = standard_df.groupby('mri').slices.max()
ar

mri
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-101_space-T2S_desc-masked_T2S.nii.gz    34
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-102_space-T2S_desc-masked_T2S.nii.gz    34
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-103_space-T2S_desc-masked_T2S.nii.gz    34
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-104_space-T2S_desc-masked_T2S.nii.gz    34
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-105_space-T2S_desc-masked_T2S.nii.gz    34
                                                                                                                             ..
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-323_space-T2S_desc-masked_T2S.nii.gz    38
C:\Users\araza\Documents\1\gits\thesis_project\Dataset\VALDO_dataset\mri_t2s_dir\sub-324_space-T2S_d

### Check if all slices match with the raw dataframe count

In [9]:
(ar == (arr - 1)).all()

True

### Export as metadata

In [10]:
df.to_csv('targets.csv', index=False)